# MAGICAL: Operational Amplifier in Skywater 130nm

--------

MAHICAL Team, November 2022
SPDX-License-Identifier: Apache-2.0

--------
# Category 2: A new idea that could be the basis for a future tapeout

| Name | Affiliation | IEEE Member | SSCS Member |
| ----------- | ----------- | ----------- | ----------- |
| Keren Zhu (Lead) Email: keren.zhu@utexas.edu | UT Austin | Yes | No |
| Hao Chen | UT Austin | Yes | No |
| David Pan (Supervisor) | UT Austin | Yes | No |

MAGICAL is initially developed under TSMC 40nm. We recently develop the Skywater 130nm branch to support the open-source PDK. This demostration of amplifier is preliminary and in early stage. We hope we can extend the tool and this notebook can be the basis for a potential future tapeout.

--------
# Background

This notebook serves as an introduction to MAGICAL, an open-source fully-automated analog layout synthesis software, in the open-source Skywater 130nm. MAGICAL is a standalone general-purpose analog layout generation framework using placement and routing algorithms. MAGICAL is previously verified in silicon with data converter designs in TSMC 40nm technology[1]. We recently extend the software capability to support Skywater 130nm PDK. 

MAGICAL includes a complete software stack including device generation, constraint generation, placement and routing. The software is developed in Python and C++, where Python serves as the top-level interface and C++ is used for computational-intense tasks.

[1] Hao Chen∗, Mingjie Liu∗, Xiyuan Tang∗, Keren Zhu∗, Abhishek Mukherjee, Nan Sun and David Z. Pan, “MAGICAL 1.0: An Open-Source Fully-Automated AMS Layout Synthesis Framework Verified With a 40- nm 1 GS/s ∆Σ ADC,” in IEEE Custom Integrated Circuits Conference (CICC), Virtual Event, April 25-30, 2021. (∗ indicates equal contributions in alphabetic order)

--------
# Dependencies and Software Installation

MAGICAL need to be installed to Python environment with several C++ and Python dependencies.

This notebook is run under a docker environment at [https://github.com/krzhu/MAGICAL-docker](https://github.com/krzhu/MAGICAL-docker)

A user can follow the Dockerfile when building the software if not using Docker.
```
FROM jupyter/scipy-notebook
LABEL maintainer="Keren Zhu <keren.zhu@utexas.edu>"

# Switch to root user
USER root 


# Install cmake 3.16.3
ADD https://cmake.org/files/v3.16/cmake-3.16.3-Linux-x86_64.sh /cmake-3.16.3-Linux-x86_64.sh
RUN mkdir /opt/cmake \
        && sh /cmake-3.16.3-Linux-x86_64.sh --prefix=/opt/cmake --skip-license \
        && ln -s /opt/cmake/bin/cmake /usr/local/bin/cmake \
        && cmake --version

# Update apt packages
RUN apt update
RUN apt upgrade -y

# Install boost flex
RUN apt-get install -y \
    build-essential zlib1g-dev libncurses5-dev libnss3-dev libssl-dev \
    libreadline-dev libffi-dev \
    libboost-all-dev flex csh vim \
    python3-openssl 
    
# Update openssl
RUN apt-get remove -y openssl
RUN wget https://www.openssl.org/source/openssl-1.1.1d.tar.gz --no-check-certificate \
        && tar -zxf openssl-1.1.1d.tar.gz && cd openssl-1.1.1d \
        && ./config && make && make install \
        && ln -s /usr/local/bin/openssl /usr/bin/openssl && ldconfig \
        && openssl version
        

# Install lp_solve
RUN git clone https://github.com/krzhu/lp_solve.git  \
        && cd lp_solve/lpsolve55 \
        && sh ccc \
        && cp ~/lp_solve/lpsolve55/bin/ux64/* ~/lp_solve
ENV LPSOLVE_DIR=/home/jovyan/lp_solve

# Install lemon
RUN wget lemon.cs.elte.hu/pub/sources/lemon-1.3.1.tar.gz --no-check-certificate
RUN tar -xf lemon-1.3.1.tar.gz \
        && mkdir lemon-1.3.1/build && cd lemon-1.3.1/build \ 
        && cmake -DCMAKE_INSTALL_PREFIX=~/lemon .. \
        && make && make install 
ENV LEMON_DIR=/home/jovyan/lemon


# Install eigen
RUN wget https://gitlab.com/libeigen/eigen/-/archive/3.3.3/eigen-3.3.3.tar.gz --no-check-certificate \
        && tar -xf eigen-3.3.3.tar.gz 
ENV EIGEN_INC=/home/jovyan/eigen-3.3.3

# Install pybind11
RUN git clone https://github.com/pybind/pybind11.git
ENV PYBIND11_DIR=/home/jovyan/pybind11

#  python lib
RUN apt-get install python3-dev
RUN apt-get install -y python2-dev python2 python-dev-is-python3

# Install sparsehash
RUN wget https://github.com/sparsehash/sparsehash/archive/refs/tags/sparsehash-2.0.4.tar.gz --no-check-certificate
RUN tar -xf sparsehash-2.0.4.tar.gz \
        && cd sparsehash-sparsehash-2.0.4 \
        && ./configure --prefix=/sparsehash \
        && make && make install
ENV SPARSE_HASH_DIR=/sparsehash

# Install bison 3.4
RUN wget http://ftp.gnu.org/gnu/bison/bison-3.4.tar.gz --no-check-certificate
RUN tar -xf bison-3.4.tar.gz \
        && cd bison-3.4 \
        && ./configure && make && make install \
        && ln -s /usr/local/bin/bison /usr/bin/bison \
        && bison --version

# Install limbo
RUN git clone https://github.com/limbo018/Limbo.git
RUN perl -pi -e 's/TRUE/FTRUE/g' Limbo/limbo/solvers/api/LPSolveApi.h # Work around the legacy issue of lpsolve
RUN mkdir Limbo/build && cd Limbo/build \
        && cmake .. -DCMAKE_INSTALL_PREFIX=/limbo \
        && make && make install
ENV LIMBO_DIR=/limbo
ENV LIMBO_INC=/limbo/include

# Install python prerequisite
RUN pip install networkx matplotlib scipy numpy Cython pybind11
RUN git clone https://github.com/jayl940712/gdspy.git \
        && pip install gdspy/

# Install Boost
RUN apt-get install libboost-all-dev

# Install magical
RUN mkdir install && cd install && git clone https://github.com/magical-eda/MAGICAL.git \
        && cd MAGICAL && git checkout docker \
        && git submodule init && git submodule update \
        && cd /home/jovyan/install/MAGICAL/device_generation && git checkout sky130 \
        && cd /home/jovyan/install/MAGICAL\
        && git checkout skywater \
        && ./build.sh
``` 


# Input Netlist

Here we use a simple OTA design to demonstrate our flow.

In [1]:
%cd ~
!git clone https://github.com/magical-eda/MAGICAL.git
%cd MAGICAL
!git checkout skywater


/home/jovyan
Cloning into 'MAGICAL'...
remote: Enumerating objects: 8367, done.
remote: Counting objects: 100% (1051/1051), done.
remote: Compressing objects: 100% (416/416), done.
remote: Total 8367 (delta 590), reused 1023 (delta 581), pack-reused 7316
Receiving objects: 100% (8367/8367), 21.88 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (3754/3754), done.
/home/jovyan/MAGICAL
Branch 'skywater' set up to track remote branch 'skywater' from 'origin'.
Switched to a new branch 'skywater'


In [1]:
%cd ~
!mkdir ota_example

%cd ~/ota_example

/home/jovyan
/home/jovyan/ota_example


In [2]:


netlist = "subckt OTA_2 GND IBIAS VCM VDD VIM VIP VOM VOP \n \
    M36 (net0134 PCAS VDD VDD) pch l=200n w=2.4u multi=1 nf=2 \n \
    C7 (vtail VOP) cap l=3u w=3u stm=3 \n \
    C6 (vtail VOM) cap l=3u w=3u stm=3 \n \
    C5 (VOM net0101) cap l=3u w=3u stm=3 \n \
    C4 (VOP net0101) cap l=3u w=3u stm=3 \n \
    C1 (VO1P net0118) cap l=8u w=6u stm=3 \n \
    C0 (VO1M net0118) cap l=8u w=6u stm=3 \n \
    M33 (net0138 net077 VDD VDD) pch l=200n w=4.8u multi=1 nf=4 \n  \
    M32 (net0136 net077 VDD VDD) pch l=200n w=7.2u multi=1 nf=6 \n  \
    M21 (net0104 net0101 CMFBTAIL GND) nch_lvt l=200n w=5.4u multi=1 nf=6 \n  \
    M20 (vtail VCM CMFBTAIL GND) nch_lvt l=200n w=5.4u multi=1 nf=6 \n  \
    M18 (VO1P IBIAS net0131 GND) nch_lvt l=240.0n w=118.8u multi=1 nf=33 \n  \
    M17 (VO1M IBIAS net0133 GND) nch_lvt l=240.0n w=118.8u multi=1 nf=33 \n  \
    M7 (CMFBTAIL NBIAS_TAIL GND GND) nch_lvt l=200n w=11.7u multi=1 nf=13 \n  \
    M8 (net0133 VIP NTAIL GND) nch_lvt l=240.0n w=115.2u multi=1 nf=32 \n  \
    M9 (VOM net092 vs GND) nch_lvt l=200n w=108.000000u multi=1 nf=30 \n  \
    M10 (net0131 VIM NTAIL GND) nch_lvt l=240.0n w=115.2u multi=1 nf=32 \n  \
    M12 (VOP net096 vs GND) nch_lvt l=200n w=108.000000u multi=1 nf=30 \n  \
    M1 (NTAIL NBIAS_TAIL GND GND) nch_lvt l=200n w=93.6u multi=1 nf=52 \n \
    M19 (IBIAS IBIAS NBIAS_TAIL GND) nch_lvt l=200n w=9.9u multi=1 nf=11 \n  \
    M16 (PCAS VCM VN1 GND) nch_lvt l=200n w=11.7u multi=1 nf=13 \n  \
    M15 (NBIAS_TAIL VCM VN2 GND) nch_lvt l=200n w=11.7u multi=1 nf=13 \n  \
    M14 (INCM2 INCM2 net0137 GND) nch_lvt l=240.0n w=1.2u multi=1 nf=2 \n  \
    M13 (net077 VCM net0135 GND) nch_lvt l=200n w=4.5u multi=1 nf=5 \n  \
    M3 (net0135 NBIAS_TAIL GND GND) nch_lvt l=200n w=4.5u multi=1 nf=5 \n  \
    M4 (VN2 NBIAS_TAIL GND GND) nch_lvt l=200n w=11.7u multi=1 nf=13 \n  \
    M2 (VN1 NBIAS_TAIL GND GND) nch_lvt l=200n w=11.7u multi=1 nf=13 \n  \
    M74 (PTAIL net0118 VDD VDD) pch l=200n w=48.0u multi=1 nf=40 \n  \
    M71 (VOP VO1M VDD VDD) pch l=200n w=48.0u multi=1 nf=20 \n  \
    M70 (VOM VO1P VDD VDD) pch l=200n w=48.0u multi=1 nf=20 \n  \
    M69 (net0104 net0104 VDD VDD) pch l=200n w=7.2u multi=1 nf=6 \n  \
    M68 (vtail net0104 VDD VDD) pch l=200n w=7.2u multi=1 nf=6 \n  \
    M35 (net0132 VIP PTAIL VDD) pch l=240.0n w=86.4u multi=1 nf=24 \n  \
    M34 (net0130 VIM PTAIL VDD) pch l=240.0n w=86.4u multi=1 nf=24 \n  \
    M27 (VO1P PCAS net0130 VDD) pch l=240.0n w=124.8u multi=1 nf=26 \n  \
    M26 (VO1M PCAS net0132 VDD) pch l=240.0n w=124.8u multi=1 nf=26 \n  \
    M28 (PCAS PCAS net0134 VDD) pch l=200n w=1.2u multi=1 nf=1 \n  \
    M25 (INCM2 PCAS net0138 VDD) pch l=200n w=4.8u multi=1 nf=4 \n  \
    M24 (net077 PCAS net0136 VDD) pch l=200n w=7.2u multi=1 nf=6 \n  \
    M50 (vs vtail GND GND) nch_lvt l=200n w=144.000000u multi=1 nf=40 \n  \
    M6 (net0137 INCM2 GND GND) nch_lvt l=240.0n w=1.2u multi=1 nf=2 \n  \
    R11 (net0101 VOM) rpoly lr=15u wr=0.4u series=6 \n \
    R14 (net0101 VOP) rpoly lr=15u wr=0.4u series=6 \n \
    R5 (VO1M net0118) rpoly lr=20u wr=0.4u series=18 \n \
    R12 (VO1P net0118) rpoly lr=20u wr=0.4u series=18 \n \
    R13 (net096 INCM2) rpoly lr=20u wr=0.4u series=10 \n \
 \n \
ends OTA_2"
with open("ota.sp", "w") as sp_file:
    sp_file.write(netlist)

# Prepare Input Parameters

Here we specify the technology files and symmetry constraints.

In [8]:
jsonfile = "{ \n \
    \"spectre_netlist\" : \"ota.sp\", \n \
    \"resultDir\" : \"./\", \n \
    \"techfile\"  : \"../MAGICAL/examples/skywaterPDK/sky130.techfile\", \n \
    \"simple_tech_file\" :  \"../MAGICAL/examples/skywaterPDK/sky130.simple\", \n \
    \"lef\" : \"../MAGICAL/examples/skywaterPDK/sky130.lef\", \n \
    \"gds_layer_mapping\" : \"../MAGICAL/examples/skywaterPDK/layermapping.layermapping\" \n \
} \n \
"
with open("ota.json", "w") as j_file:
    j_file.write(jsonfile)
    
    
sym = "OTA_2_M21 OTA_2_M20 \n \
OTA_2_M69 OTA_2_M68 \n \
OTA_2_M7 \n \
OTA_2_M8 OTA_2_M10 \n \
OTA_2_M17 OTA_2_M18 \n \
OTA_2_M71 OTA_2_M70 \n \
OTA_2_M12 OTA_2_M9 \n \
OTA_2_C4 OTA_2_C5 \n \
OTA_2_C7 OTA_2_C6 \n \
OTA_2_M26 OTA_2_M27 \n \
OTA_2_M35 OTA_2_M34 \n \
OTA_2_C0 OTA_2_C1 \n \
OTA_2_M1 \n \
OTA_2_M50 \n \
OTA_2_M74 \n \
OTA_2_R14 OTA_2_R11 \n \
OTA_2_R5 OTA_2_R12 \n \
OTA_2_M24 OTA_2_M32 \n \
OTA_2_M13 OTA_2_M3 \n \
OTA_2_M25 OTA_2_M33 \n \
OTA_2_R13 \n \
OTA_2_M2 OTA_2_M4 \n \
"

with open("OTA_2.sym", "w") as sym_file:
    sym_file.write(sym)
    
    
symnet="net0133 net0131 \n\
VO1M VO1P \n\
VOP VOM \n\
net0132 net0130 \n\
CMFBTAIL \n\
NTAIL \n\
vs \n\
PTAIL \n\
net0118"

with open("OTA_2.symnet", "w") as symnet_file:
    symnet_file.write(symnet)
    

# Layout Generation

After setting the input files, running MAGICAL for analog circuit is easy with one line of command.

In [9]:
!mkdir gds


!python ~/MAGICAL/flow/python/Magical.py ota.json

mkdir: cannot create directory ‘gds’: File exists
/opt/conda/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
    MAGICAL: Machine Generated Analog IC Layout            
        https://github.com/magical-eda/M

# Show Results

After the placement and routing, MAGICAL has generated the GDS for the OTA design.

In [23]:
!pip install gdstk svgutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.7 MB/s eta 0:00:00a 0:00:01


In [25]:


import gdstk
import IPython.display
import svgutils.transform as sg

flrpln = gdstk.read_gds("OTA_2_flat.gds")
flrpln_top_cell = flrpln.top_level()
flrpln_top_cell[0].write_svg('final.svg')
fig = sg.fromfile('final.svg')
fig.set_size(('700','700'))
fig.save('final.svg')
IPython.display.SVG('final.svg')

